In [1]:
import os
import glob

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Render plots as SVG
%config InlineBackend.figure_format = 'svg'

In [2]:
meta = pd.read_excel('data/metadata.xlsx')
meta.sample(5)

,File name,Author,Title,Publication date,Series + n°,Editor,Illustrator,Publisher,Place of publication,Intended reader,Source intended reader,Date of debut,Author's birth date,Author's age at time of publication,Author's gender,Country,Comments,English title
729,WOLTZ_alaska_2016,Anna Woltz,Alaska,2016,NaN,NaN,NaN,Querido,Amsterdam,11,website author,1998,1981,35,F,NL,NaN,NaN
511,VANDERGEEST_spinder_2012,Simon van der Geest,Spinder,2012,NaN,NaN,Karst-Janneke Rogaar,Querido,Amsterdam,10,website author,2009,1978,34,M,NL,NaN,NaN
639,WILSON_bigdayout_2012,Jacqueline Wilson,Big Day Out,2012,NaN,NaN,Nick Sharratt,Corgi,London,9,ama,1969,1945,67,F,UK,NaN,NaN
406,MURPHY_theworstwitchsavestheday_2005,Jill Murphy,The Worst Witch Saves the Day,2005,The Worst Witch #5,NaN,Jill Murphy,Puffin,London,8,CBK,1973,1949,56,F,UK,NaN,NaN
271,GAVIN_dangerbymoonlight_2004,Jamila Gavin,Danger by Moonlight,2004,NaN,NaN,NaN,Egmont,London,7,guardian,1979,1941,63,F,UK,NaN,NaN


In [3]:
meta.shape

(753, 18)

In [4]:
meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   File name                            753 non-null    object
 1   Author                               753 non-null    object
 2   Title                                753 non-null    object
 3   Publication date                     753 non-null    int64 
 4   Series  + n°                         167 non-null    object
 5   Editor                               1 non-null      object
 6   Illustrator                          410 non-null    object
 7   Publisher                            741 non-null    object
 8   Place of publication                 739 non-null    object
 9   Intended reader                      753 non-null    object
 10  Source intended reader               643 non-null    object
 11  Date of debut                        753 non-

In [5]:
memory_before = meta.memory_usage(deep=True).sum() / 1024
f'{memory_before:.2f} KB'

'596.49 KB'

## Cleaning up the dataset

(following [Matt Harrison's](https://twitter.com/__mharrison__) 'chaining' approach to Pandas)

In [6]:
meta = (
    meta
    .rename(lambda x: x.replace(' ', '_').lower(), axis=1)
    .rename(columns={'publication_date': 'pub_year', "author's_age_at_time_of_publication": 'author_age_pub', "author's_gender": 'author_gender'})
    .loc[:, ['file_name', 'author', 'title', 'pub_year', 'intended_reader', 'author_age_pub', 'author_gender', 'country']]
    .dropna(axis=0, how='any')
    .query(
        "country == 'UK' & \
        intended_reader != '-' & \
        author.str.contains('/') == False"
    )
    .astype({
        'author': 'category',
        'intended_reader': int,
        'author_gender': 'category'
    })
)

meta.sample(10)

,file_name,author,title,pub_year,intended_reader,author_age_pub,author_gender,country
70,BLACKMAN_pigheartboy_1997,Malorie Blackman,Pig Heart Boy,1997,9,35,F,UK
56,BLACKMAN_hacker_1992,Malorie Blackman,Hacker,1992,9,30,F,UK
487,ROWLING_thecuckooscalling_2013,J.K. Rowling,The Cuckoo's Calling,2013,18,48,F,UK
425,PULLMAN_iwasarat_1999,Philip Pullman,I was a Rat!,1999,9,53,M,UK
150,FINE_holeintheroad_2014,Anne Fine,Hole in the Road,2014,9,67,F,UK
663,WILSON_jackydaydream_2007,Jacqueline Wilson,Jacky Daydream,2007,9,62,F,UK
128,FINE_apackofliars_1988,Anne Fine,A Pack of Liars,1988,9,41,F,UK
48,BLACKMAN_elaineyoureabrat_1991,Malorie Blackman,Elaine You're a Brat,1991,5,29,F,UK
691,WILSON_secrets_2002,Jacqueline Wilson,Secrets,2002,9,57,F,UK
489,ROWLING_thesilkworm_2014,J.K. Rowling,The Silkworm,2014,18,49,F,UK


In [7]:
meta = (meta
    .assign(reader_age_group=np.where(meta['intended_reader'] == 18, 'adult' , 'child'))
    .astype({'reader_age_group': 'category'})
    .drop(['country', 'intended_reader'], axis=1)
)

In [8]:
meta.shape

(327, 7)

In [9]:
memory_after = meta.memory_usage(deep=True).sum() / 1024
f'{memory_after:.2f} KB'

'61.35 KB'

In [10]:
# memory difference
print(f'{abs(memory_after - memory_before):.2f} KB')

535.15 KB


In [11]:
meta.groupby('reader_age_group')['author'].value_counts(sort=False)

reader_age_group                      
adult             Aidan Chambers           0
                  Anne Fine                9
                  David Almond             2
                  Frank Cottrell-Boyce     0
                  J.K. Rowling             6
                  Jacqueline Wilson        0
                  Jamila Gavin             0
                  Jill Murphy              0
                  Malorie Blackman         0
                  Neil Gaiman              6
                  Philip Pullman           3
child             Aidan Chambers          11
                  Anne Fine               74
                  David Almond            22
                  Frank Cottrell-Boyce    13
                  J.K. Rowling             8
                  Jacqueline Wilson       69
                  Jamila Gavin            13
                  Jill Murphy             10
                  Malorie Blackman        50
                  Neil Gaiman              8
                

In [12]:
meta['reader_age_group'].value_counts(sort=False)

adult     26
child    301
Name: reader_age_group, dtype: int64